# OCW, MATH: Why low acc?
> Possible flaws in
> * equivalence function? (checked innocent)
> * parsing? (**suspicious here.**)

In [3]:
import jsonlines as jsl
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


MATH_RESULT = "/Users/seonils/dev/rims_minimal/src/seonil_scripts/0_RESULTS_v1/math_full_0613long/chatgpt0613long_model_selection3_math_merged.jsonl"
OCW_RESULT = "/Users/seonils/dev/rims_minimal/src/seonil_scripts/0_RESULTS_v1/ocw_0613long/chatgpt0613long_model_selection3_ocw.jsonl"

In [4]:
math_df = pd.DataFrame(jsl.open(MATH_RESULT))
ocw_df = pd.DataFrame(jsl.open(OCW_RESULT))

In [5]:
math_df.shape, ocw_df.shape
math_df.columns

Index(['question', 'level', 'type', 'solution', 'answer', 'ansmap', 'solmap',
       'selection_or_rims', 'majority_ans', 'prompt_file', 'inference_mode'],
      dtype='object')

In [6]:
from utils.llm_query_utils import extract_ans_from_cot_MATHnOCW
from utils.math_util import is_equiv, is_equiv_ocw, normalize_final_answer

def diff_oldparse_newparse(df:pd.DataFrame):
    df["newpred_cot"] = df.solmap.apply(lambda d: extract_ans_from_cot_MATHnOCW(d["cot"]))
    df["oldpred_cot"] = df.ansmap.apply(lambda d: d["cot"])
    df["cot_changed"] = df.newpred_cot != df.oldpred_cot
    return df 

In [7]:
math_df_ = diff_oldparse_newparse(math_df)
ocw_df_ = diff_oldparse_newparse(ocw_df)


math_df_.cot_changed.sum(), len(math_df_), ocw_df_.cot_changed.sum(), len(ocw_df_)

(4996, 4996, 272, 272)

## oldparse vs newparse of CoT solutions
- I guess the parsing is done as our wish. 
- But the solution did not (fewshot-inferenced by gsm fewshots, thus old parsing function will work better on it)

In [8]:
math_df_.loc[:, ["newpred_cot", "oldpred_cot", "answer"]]

,newpred_cot,oldpred_cot,answer
0,$x=2$,3.0,2
1,$130\%$,1.0,10
2,$\frac{8}{7}$,8.0,\dfrac{9}{7}
3,$i^5 + i^{-25} + i^{45} = i + (-i) + i = 0$,0.0,i
4,$2^8 = 2^{2x}$,4.0,4
...,...,...,...
4991,"$\n\nSo, we can simplify the equation to:\n$",NaN,-\frac{4}{3}
4992,$\boxed{\begin{pmatrix} 13 \\ -2 \\ 3 \end{pma...,3.0,\begin{pmatrix} 1 \\ 2 \\ -3 \end{pmatrix}
4993,$\mathbf{a} + \mathbf{b} + \mathbf{c} = \mathb...,2.0,-\frac{155}{2}
4994,$\dfrac{-\sin{6^{\circ}}+\cos{6^{\circ}}}{\cos...,19.0,159


In [9]:
ocw_df_.loc[:, ["newpred_cot", "oldpred_cot"]]


,newpred_cot,oldpred_cot
0,6.5,NaN
1,$L = 4\pi (10^9 \mathrm{~cm})^2 (5.67 \times 1...,5.00
2,$41.81^\circ$,41.81
3,$m = 5 \log_{10}(3) + 18$,18.00
4,"\( 5.9 \, \mathrm{m/s^2} \)",5.90
...,...,...
267,"\(4.571 \times 10^{-25} \, \mathrm{m}\)",25.00
268,\[\n\hat{E} \psi_1 = E_1 \psi_1 \quad \text{an...,1.00
269,$\psi_2$,2.00
270,$ and $,NaN


In [14]:
def before_and_after_accuracy(df, equiv_f:callable=None):
    newparse_correct = df.progress_apply(lambda row: equiv_f(row.newpred_cot, row.answer), axis="columns")
    oldparse_correct = df.progress_apply(lambda row: equiv_f(row.oldpred_cot, row.answer), axis="columns")
    results = {
        "new_acc": round(newparse_correct.mean(),3),
        "old_acc": round(oldparse_correct.mean(),3),
        "delta correct": (newparse_correct.sum()-oldparse_correct.sum(), len(newparse_correct)),
    } 
    return results
math_eq = lambda x,y: is_equiv(normalize_final_answer(str(x)), normalize_final_answer(str(y)))
ocw_eq_sym = lambda x,y: is_equiv_ocw(str(x), str(y), use_sym_exp_normalizer=True)
ocw_eq_norm = lambda x,y: is_equiv_ocw(str(x), str(y), use_sym_exp_normalizer=False)

In [12]:
math_res = before_and_after_accuracy(math_df_, math_eq) 

100%|██████████| 4996/4996 [00:17<00:00, 284.79it/s]


In [15]:
ocw_res_norm = before_and_after_accuracy(ocw_df_, ocw_eq_norm)
ocw_res_sym = before_and_after_accuracy(ocw_df_, ocw_eq_sym)

  5%|▍         | 13/272 [00:00<00:02, 124.21it/s]

failed to parse 41.8000000000000 with exception 'Float' object is not iterable
failed to parse 41.81**circ with exception 'Pow' object is not iterable
Couldn't subtract None and None with exception unsupported operand type(s) for -: 'NoneType' and 'NoneType'
I don't understand this
\( 5.9 \, \mathrm{m/s^2} \)
~~~~~~~~~~~~~~~~~~~~~~~~~^
failed to parse 0.00600000000000000 with exception 'Float' object is not iterable
Couldn't subtract i*(n*(v*(a*(l*(i*(da*(n*(s*(w*(e*r)))))))))) and None with exception unsupported operand type(s) for -: 'Mul' and 'NoneType'
failed to parse 300 with exception 'Integer' object is not iterable
failed to parse 2.997*((k*m)/s) with exception 'Mul' object is not iterable
Couldn't subtract None and None with exception unsupported operand type(s) for -: 'NoneType' and 'NoneType'
failed to parse 0.00300000000000000 with exception 'Float' object is not iterable
failed to parse 0.003*(mathrm*(a*(r*(c*(s*(e*(c*(o*(ds*n))))))))) with exception 'Mul' object is not it

 10%|▉         | 26/272 [00:00<00:02, 91.14it/s] 

failed to parse 6630 with exception 'Integer' object is not iterable
failed to parse 65.65*AA with exception 'Mul' object is not iterable
Couldn't subtract None and None with exception unsupported operand type(s) for -: 'NoneType' and 'NoneType'
failed to parse 7*(e*37) with exception 'Mul' object is not iterable
failed to parse L with exception 'Symbol' object is not iterable
Couldn't subtract None and None with exception unsupported operand type(s) for -: 'NoneType' and 'NoneType'
failed to parse (M**(1/3)*a)/(G*M**(2/3) + b) with exception 'Mul' object is not iterable
failed to parse rho with exception 'Symbol' object is not iterable
Couldn't subtract None and None with exception unsupported operand type(s) for -: 'NoneType' and 'NoneType'


 17%|█▋        | 45/272 [00:00<00:03, 60.32it/s]

...

 47%|████▋     | 129/272 [00:01<00:02, 49.19it/s]

Couldn't subtract 1.00000000000000 and 1/(-a + b) <= -1/e**(b*t) + e**(-t) with exception unsupported operand type(s) for -: 'Float' and 'LessThan'


 90%|█████████ | 245/272 [00:02<00:00, 218.82it/s]

Couldn't subtract 2.00000000000000 and c**2*m_{p} <= (gamma**2 - 1)*sin(theta)**2 with exception unsupported operand type(s) for -: 'Float' and 'LessThan'


100%|██████████| 272/272 [00:02<00:00, 116.18it/s]


In [16]:
math_res, ocw_res_sym, ocw_res_norm

({'new_acc': 0.265,
  'old_acc': 0.247,
  'newparse_correct': (1325, 4996),
  'oldparse_correct': (1232, 4996)},
 {'new_acc': 0.191, 'old_acc': 0.099, 'delta correct': (25, 272)},
 {'new_acc': 0.195, 'old_acc': 0.099, 'delta correct': (26, 272)})

In [18]:
{'new_acc': 0.265,
  'old_acc': 0.247,
  'newparse_correct': (93, 4996),} # math
{'new_acc': 0.195, 'old_acc': 0.099, 'delta correct': (26, 272)} # ocw

{'new_acc': 0.195, 'old_acc': 0.099, 'delta correct': (26, 272)}